### Setting up notebook 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
from IPython.core.display import display, HTML
import d3_lib
from string import Template
import pandas as pd
import json
import random
import pickle
#from functions import *

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\kelihjoll\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [60]:
bakgrunnsuppl = {'hjuskaparstada': 2, #'1: einhleypur, 2: einst. foreldri, 3: giftur/sambud '
                    'busetuform': 1, #'1: eigin, 2: leigu/busetu, 3: hvorugt '
                    'tekjur': [600000, 0, 0, 0], #[laun_a_manudi, fjarmagnstekjur, laun_maka, fjarmagnstekjur_maka]
                    'eignir': 0,
                    'husnaedislan': 0,
                    'vaxtagjold': 0,
                    'husnaediskostnadur': 0,
                    'fjoldi_heimilismanna': 0,
                    'heimilistekjur': 0,
                    'heimiliseignir': 0,
                    'fjoldi_barna': 2,
                    'fjoldi_barna_undir_7': 1,
                    'idgjald': 0.04,
                    'sereignaridgjald': 0.02,
                    'serstakar': 0
                    }

In [63]:
litill_islendingur(forsendur_dict, bakgrunnsuppl)

tekjur fyrir skatt: 600000
tekjur eftir skatt: 412105
staðgreiðsla: 151894
greiðsla í lífeyrissjóð: 24000
greiðsla í viðbótarlífeyrissparnað: 12000
barnabætur á mánuði: 43458


In [51]:
def litill_islendingur(forsendur_dict, bakgrunnsuppl):
    if bakgrunnsuppl['tekjur'] != 0:
        tekjur_e_skatt, stadgreidsla, lifeyrisgreidsla, sereignarlifeyrisgr = \
        stadgreidsla_func(forsendur_dict, bakgrunnsuppl)
        print 'tekjur fyrir skatt: %d' %bakgrunnsuppl['tekjur'][0]
        print 'tekjur eftir skatt: %d' %tekjur_e_skatt
        print 'staðgreiðsla: %d' %stadgreidsla
        print 'greiðsla í lífeyrissjóð: %d' %lifeyrisgreidsla
        print 'greiðsla í viðbótarlífeyrissparnað: %d' %sereignarlifeyrisgr
        
    if bakgrunnsuppl['fjoldi_barna'] != 0:
        barnabaetur = barnabaetur_func(forsendur_dict, bakgrunnsuppl)#, ororka_skattgrunnur, elli_skattgrunnur)
        print 'barnabætur á mánuði: %d' %barnabaetur

In [37]:
def stadgreidsla_func(forsendur, info):
    
    '''
    Reiknar staðgreiðslu útfrá gefnum forsendum.

    Skilar: launum eftir skatt (int), staðgreiðslu (int),
            skyldulífeyri (int), séreignarlífeyri (int)
    '''
    
    skattthrep = forsendur['stadgr_threp_m']
    skatthlutfall_nedra = forsendur['stadgr_hlutfall_nedra']
    skatthlutfall_efra = forsendur['stadgr_hlutfall_efra']
    personuafsl = forsendur['personuafslattur_m']

    lifeyrir = (info['tekjur'][0] * info['idgjald'])
    serlifeyrir = (info['tekjur'][0] * info['sereignaridgjald'])

    stofn = info['tekjur'][0] - lifeyrir - serlifeyrir

    if stofn <= skattthrep:
        skattstofn_1 = stofn
        skattstofn_2 = 0
    else:
        skattstofn_1 = skattthrep
        skattstofn_2 = stofn - skattthrep

    skattur_nedra_threp = (skattstofn_1*skatthlutfall_nedra)
    skattur_efra_threp = (skattstofn_2*skatthlutfall_efra)

    reiknud_stadgreidsla = skattur_nedra_threp + skattur_efra_threp
    stadgreidsla_alls = reiknud_stadgreidsla - personuafsl
    if stadgreidsla_alls < 0:
        stadgreidsla_alls = 0

    laun_e_skatt = stofn - stadgreidsla_alls

    return laun_e_skatt, stadgreidsla_alls, lifeyrir, serlifeyrir

In [ ]:
def husnaedisstudningur_func(forsendur, info):#, ororka, ellilif):
    ''' Kallar á rétt húsnæðisstuðningsfall eftir búsetuformi í forsendum '''

    if info['busetuform'] == 1:
        studningur, tekjuskerding, eignaskerding = vaxtabaetur_func(forsendur, info)#, ororka, ellilif)

    elif info['busetuform'] == 2:
        studningur, tekjuskerding, eignaskerding, heimilistekjur = husnaedisbaetur_func(forsendur, info)#, ororka, ellilif)
        if info['serstakar'] == 1:
            serstakar, tekjuskerding_serstakar = serstakar_func(forsendur, info, studningur) #ororka, ellilif, studningur)
            studningur += serstakar
            tekjuskerding += tekjuskerding_serstakar

    else:
        studningur = 0

    return studningur, tekjuskerding, eignaskerding

In [58]:
def barnabaetur_func(forsendur, info):#, ororka, ellilif):
    '''
    Reiknar barnabætur út frá gefnum forsendum

    Skilar mánaðarlegum barnabótum (int)'''

    tekjur_yearly = [info['tekjur'][0]*12,
                     info['tekjur'][1]*12,
                     info['tekjur'][2]*12,
                     info['tekjur'][3]*12]
    #ororka_yearly = 12*ororka
    #ellilif_yearly = 12*ellilif

    fj_barna = info['fjoldi_barna']
    fj_barna_undir_7 = info['fjoldi_barna_undir_7']

    # SKERÐINGARHLUFÖLL
    skerdingarhlutfoll = forsendur['barnab_skerdingarhlutfoll'] # [1 barn, 2 börn, 3 börn eða fleiri]
    skerdingarhlutf_vidbot = forsendur['barnab_skerdingarhl_vidbot']

    # FJÁRHÆÐIR 2017
    fjarhaedir_fyrsta_barn = forsendur['barnab_fjarh_fyrsta_barn_y'] # [einstætt foreldri, hjón/sambúðarfólk]
    fjarhaedir_umfram_born = forsendur['barnab_fjarh_umfram_born_y'] # [einstætt foreldri, hjón/sambúðarfólk]
    vidbotarfjarhaed_hvert_barn_undir_7 = forsendur['barnab_fjarh_barn_undir_7_y']

    # SKERÐINGARMÖRK
    skerdingarmork = forsendur['barnab_tekjuskmork'] # [einstætt foreldri, hjón/sambúðarfólk]

    # ÚTREIKNINGUR ÓSKERTRAR BÓTAFJÁRHÆÐAR
    stofn_til_skerdingar = sum(tekjur_yearly) - skerdingarmork[info['hjuskaparstada']-2] #+ ororka_yearly + ellilif_yearly
    if stofn_til_skerdingar < 0:
        stofn_til_skerdingar = 0

    almennar_barnabaetur =\
        fjarhaedir_fyrsta_barn[info['hjuskaparstada']-2] +\
        ((fj_barna-1) * fjarhaedir_umfram_born[info['hjuskaparstada']-2])

    vidbotarbaetur = (fj_barna_undir_7 * vidbotarfjarhaed_hvert_barn_undir_7)

    skerding = (skerdingarhlutfoll[fj_barna-1] * stofn_til_skerdingar)
    if skerding > almennar_barnabaetur:
        skerding = almennar_barnabaetur
    skerding_vidbotar = ((fj_barna_undir_7*skerdingarhlutf_vidbot) * stofn_til_skerdingar)
    if skerding_vidbotar > vidbotarbaetur:
        skerding_vidbotar = vidbotarbaetur

    baetur = almennar_barnabaetur + vidbotarbaetur - (skerding + skerding_vidbotar)
    if baetur < 0:
        baetur = 0

    if info['hjuskaparstada'] == 2:
        return baetur/12
    else:
        return baetur/24 #bætur deilast jafnt milli hjóna

In [ ]:
def vaxtabaetur_func(forsendur, info):#, ororka, ellilif):
    '''
    Reiknar vaxtabætur út frá gefnum forsendum.

    Skilar: vaxtabótum (int), tekjuskerðingu (int) og eignaskerðingu (int)
    '''

    hamark_vaxtagjalda = forsendur['hamark_vaxtagjalda_y'] #fyrir: [einstakling, einstætt foreldri, hjón]
    hamark_bota = forsendur['hamark_vaxtabota_y'] #fyrir: [einstakling, einstætt foreldri, hjón]
    tekjur_yearly = [info['tekjur'][0]*12,\
                     info['tekjur'][1]*12,\
                     info['tekjur'][2]*12,\
                     info['tekjur'][3]*12]
    ororka_yearly = 12*ororka
    ellilif_yearly = 12*ellilif
    tekjur = sum(tekjur_yearly) + ororka_yearly + ellilif_yearly

    eignaskerdingarmork_nedri = forsendur['vaxtab_eignask_nedri'] #fyrir: [einstakling, einstætt foreldri, hjón]
    eignaskerdingarmork_efri = forsendur['vaxtab_eignask_efri'] #fyrir: [einstakling, einstætt foreldri, hjón]

    stofn_list = [hamark_vaxtagjalda[info['hjuskaparstada']-1],
                  forsendur['vaxtab_hamarkshl_huslans']*info['husnaedislan'],
                  info['vaxtagjold']]
    stofn =  min(float(s) for s in stofn_list)

    tekjuskerding = forsendur['vaxtab_tekjusk_hlutfall'] * tekjur
    baetur = stofn - tekjuskerding
    if baetur < 0:
        baetur = 0

    if info['eignir'] >= eignaskerdingarmork_efri[info['hjuskaparstada']-1]:
        eignaskerding = baetur
        baetur = 0

    elif info['eignir'] >= eignaskerdingarmork_nedri[info['hjuskaparstada']-1]:
        eignaskerding =\
            baetur *\
            ((info['eignir'] - eignaskerdingarmork_nedri[info['hjuskaparstada']-1]) /
            (eignaskerdingarmork_efri[info['hjuskaparstada']-1] -
             eignaskerdingarmork_nedri[info['hjuskaparstada']-1]))
        baetur -= eignaskerding

    else:
        eignaskerding = 0

    if baetur > hamark_bota[info['hjuskaparstada']-1]:
        baetur = hamark_bota[info['hjuskaparstada']-1]

    if info['hjuskaparstada'] == 3:
        baetur = baetur/24 #bætur deilast jafnt milli hjóna
    else:
        baetur = baetur/12
    tekjuskerding = tekjuskerding/12
    eignaskerding = (eignaskerding/12)

    if baetur < 0:
        baetur = 0
    if tekjuskerding < 0:
        tekjuskerding = 0
    if eignaskerding < 0:
        eignaskerding = 0

    return baetur, tekjuskerding, eignaskerding



In [36]:
def husnaedisbaetur_func(forsendur, info, ororka, ellilif):
    '''
    Reiknar húsnæðisbætur (húsaleigubætur) út frá gefnum forsendum

    Skilar húsaleigubótum (int), tekjurskerðingu (int),
    eignaskerðingu (int) og heimilistekjum (int)
    '''

    tekjur = info['heimilistekjur'] + sum(info['tekjur']) + ororka + ellilif

    grunnfjarhaedir = forsendur['husnb_grunnur_m']
    fritekjumark = forsendur['husnb_fritekjumark_m']
    eignaskerdingarmork_nedri = forsendur['husnb_eignask_nedri']
    eignaskerdingarmork_efri = forsendur['husnb_eignask_efri']

    stofn = grunnfjarhaedir[(info['fjoldi_heimilismanna']-1)]
    heimilistekjur = info['heimilistekjur']

    if heimilistekjur <= fritekjumark[info['fjoldi_heimilismanna']-1]:
        baetur = stofn
        tekjuskerding = 0
    else:
        tekjuskerding = forsendur['husnb_tekjusk_hlutfall'] *\
        (heimilistekjur - fritekjumark[info['fjoldi_heimilismanna']-1])
        baetur = stofn - tekjuskerding

    if info['heimiliseignir'] >= eignaskerdingarmork_efri:
        eignaskerding = baetur
        baetur = 0
    elif info['heimiliseignir'] >= eignaskerdingarmork_nedri:
        eignaskerding = baetur *\
            ((info['heimiliseignir'] - eignaskerdingarmork_nedri) /
             (eignaskerdingarmork_efri - eignaskerdingarmork_nedri))
        baetur -= eignaskerding
    else:
        eignaskerding = 0

    if baetur > forsendur['husnb_hamarkshluti_husnkostn']*\
    info['husnaediskostnadur']:
        baetur = forsendur['husnb_hamarkshluti_husnkostn']*\
        info['husnaediskostnadur']
    if baetur < 0:
        baetur = 0
    if tekjuskerding < 0:
        tekjuskerding = 0
    if eignaskerding < 0:
        eignaskerding = 0

    return baetur, tekjuskerding, eignaskerding, heimilistekjur

In [62]:
forsendur_dict = {'utvarpsgjald_threshold_m': 1678001/12,
                 'utvarpsgjald_m': 16800/12,
                 'eftirlaunaaldur': 67,
                 'serstakar_tekjusk_nedri_m': [281083.0, 371755.0, 435226.0, 471495.0],
                 'serstakar_tekjusk_efri_m': [351354, 464694, 544032, 589368],
                 'serstakar_threshold_eign': 5126000,
                 'serstakar_lagmarks_leiga': 40000,
                 'serstakar_plus_venjulegar_hamark': 90000,
                 'stadgr_threp_m': 927087,
                 'stadgr_hlutfall_nedra': 0.3694,
                 'stadgr_hlutfall_efra': 0.4624,
                 'fjarmagstekjuskattur_hlutfall': 0.2,
                 'personuafslattur_m': 56447,
                 'hamark_vaxtagjalda_y': [800000, 1000000, 1200000],
                 'hamark_vaxtabota_y': [400000, 500000, 600000],
                 'vaxtab_eignask_nedri': [4500000.0, 4500000.0, 7300000.0],
                 'vaxtab_eignask_efri': [7200000.0, 7200000.0, 11680000.0],
                 'vaxtab_tekjusk_hlutfall': 0.085,
                 'vaxtab_hamarkshl_huslans': 0.07,
                 'husnb_grunnur_m': [372000/12, 492000/12, 576000/12, 624000/12],
                 'husnb_fritekjumark_m': [3373000/12, 4461064/12, 5222710/12, 5657936/12],
                 'husnb_eignask_nedri': 6500000.0,
                 'husnb_eignask_efri': 10400000.0,
                 'husnb_tekjusk_hlutfall': 0.09,
                 'husnb_hamarkshluti_husnkostn': 0.75,
                 'barnab_skerdingarhlutfoll': [0.04, 0.06, 0.08],
                 'barnab_skerdingarhl_vidbot': 0.04,
                 'barnab_fjarh_fyrsta_barn_y': [390700, 234500],
                 'barnab_fjarh_umfram_born_y': [400800, 279200],
                 'barnab_fjarh_barn_undir_7_y': 140000,
                 'barnab_tekjuskmork': [2700000, 5400000],
                 'ororka_grunnur': 44866.0,
                 'ororka_tekjusk_nedri': 214602.0,
                 'ororka_tekjusk_efri': 394066.0,
                 'ororka_fritekjumark_fjtekna_m': 98640.0/12,
                 'ororka_fritekjumark_launa_m': 1315200/12,
                 'ororka_fritekjumark_lifeyris_m': 328800/12,
                 'ororka_tekjutr_m': 143676,
                 'ororka_tekjutr_hamark_tekna_m': 376644,
                 'ororka_heimilisuppbot_m': 48564,
                 'barnalifeyrir/barn': 33168,
                 'medlag/barn': 33168,
                 'ororka_framfvidmid_ekkieinn': 238594,
                 'ororka_framfvidmid_einn': 300000,
                 'ororka_tekjutr_skerdingarhlutf_nedri': 0.3835,
                 'ororka_tekjutr_skerdingarhlutf_efri': 0.1335,
                 'ororka_heimilisuppbot_skerdingarhlutf': 0.1296,
                 'ororka_bensinstyrkur_m': 16583,
                 'foreldralaun_2born_m': 9602,
                 'foreldralaun_fleiri_born_m': 24965,
                 'ellilif_lifeyrir_m': 239484,
                 'ellilif_heimilisuppbot_m': 60516,
                 'ellilif_bensinstyrkur_m': 16583,
                 'ellilif_radstofunarfe_m': 71889,
                 'ellilif_fritekjumork': 25000,
                 'ellilif_lifeyrir_efri_mork_m': 557187,
                 'ellilif_heimilisuppbot_efri_mork_m': 533538,
                 'ellilif_lifeyrir_tekjusk_hlutf': 0.45,
                 'ellilif_huppbot_tekjusk_hlutf': 0.119,
                 'ellilif_haekkun_v_frestunar': 0.05
                }